In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [3]:
train_data_labels = np.array(train_data['label']) # labels
train_data_pixels_1d = np.array(train_data.drop(columns=['label'], axis=1))   # pixels

In [4]:
# normalize data
train_data_pixels_1d = train_data_pixels_1d / 255.0
test_data = test_data / 255.0

In [5]:
print(train_data_pixels_1d.shape)
print(test_data.shape)

(42000, 784)
(28000, 784)


In [6]:
train_data_pixels_2d = train_data_pixels_1d.reshape(train_data_pixels_1d.shape[0], 28, 28, 1)
test_data_2d = np.array(test_data).reshape(test_data.shape[0], 28, 28, 1)

In [7]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
# make label into onehot array
one_hot_label = to_categorical(train_data_labels)

Using TensorFlow backend.


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_X, test_X, train_y, test_y = train_test_split(train_data_pixels_2d, one_hot_label, random_state=0)

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D

In [20]:
# define model 
model = Sequential()

# conv2d 1
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
# conv2d 2
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# conv2d 3
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
# conv2d 5
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(256, activation = "relu"))

model.add(Dense(256, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [22]:
# Define the optimizer
from keras.optimizers import RMSprop
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# Compile the model
model.compile(optimizer =optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [23]:
batch_size = 86
epochs = 5
model.fit(train_X, train_y, 
              epochs=epochs, 
              batch_size=batch_size, 
              validation_data=(test_X, test_y))

Train on 31500 samples, validate on 10500 samples
Epoch 1/5
31500/31500 [==============================] - 165s 5ms/step - loss: 0.5454 - acc: 0.8253 - val_loss: 0.1177 - val_acc: 0.9726
Epoch 2/5
31500/31500 [==============================] - 145s 5ms/step - loss: 0.1130 - acc: 0.9724 - val_loss: 0.1450 - val_acc: 0.9657
Epoch 3/5
31500/31500 [==============================] - 145s 5ms/step - loss: 0.0815 - acc: 0.9814 - val_loss: 0.0690 - val_acc: 0.9852
Epoch 4/5
31500/31500 [==============================] - 145s 5ms/step - loss: 0.0772 - acc: 0.9825 - val_loss: 0.0601 - val_acc: 0.9845
Epoch 5/5
31500/31500 [==============================] - 145s 5ms/step - loss: 0.0713 - acc: 0.9843 - val_loss: 0.0629 - val_acc: 0.9863


In [15]:
model.save('mnist_conv2d_layer.model')

# Make Submission

In [ ]:
train_data_predictions = model.predict(test_data_2d)
train_data_predictions_numbers = [np.argmax(pred) for pred in train_data_predictions]

In [ ]:
submission_df = pd.DataFrame(columns=['ImageId', 'Label'], index=range(len(train_data_predictions)))
submission_df['Label'] = train_data_predictions_numbers
submission_df['ImageId'] = range(1, len(train_data_predictions)+1)

In [ ]:
submission_df.to_csv('./data/submission_conv2d.csv', index=False)

In [ ]:
submission_df.tail()